<a href="https://colab.research.google.com/github/NSR9/EVA8/blob/main/S4/NoteBook_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NoteBook 1:-
##Target:

* Decrese the Number of Parameters
* Create a dense structure

## Results: (to Change)
* Parameters: **5,024**
* Best Train Accuracy: **98.46 at 15th Epoch**
* Best Test Accuracy: **98.17% at 15th Epoch**

## Analysis:
* Model is overfitting.
* Model is a Slow learner Due to low Parameter count. 
* Parameter count decreased but accuracy not good enough.
* The Capacity of the Model can be increased.
* Reguralrization and BatchNormalization needs to be implemented.


In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib.pyplot as plt

import numpy as np
import seaborn as sns

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        ##Conv Block 1
        self.conv_block1 = nn.Sequential(
              nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1, bias=False), #28, 28
              nn.ReLU(),

              nn.Conv2d(in_channels=8, out_channels=8, kernel_size=3, padding=1, bias=False), #28, 28
              nn.ReLU(),
        )

        ##Transition Block 1
        self.trans1 = nn.Sequential(
              nn.MaxPool2d(2, 2),#28, 14
        )

        ##Conv Block 2        
        self.conv_block2 = nn.Sequential(
              nn.Conv2d(in_channels=8, out_channels=10, kernel_size=3, padding=0, bias=False),#14, 12
              nn.ReLU(),

              nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, padding=0, bias=False),#12, 10
              nn.ReLU(),
              
              nn.Conv2d(in_channels=10, out_channels=10, kernel_size=3, padding=0, bias=False),#10, 8
              nn.ReLU(),
              
              nn.Conv2d(in_channels=10, out_channels=16, kernel_size=3, padding=0, bias=False),#8, 6
              nn.ReLU(),
        )

        ##Global Average Pooling
        self.gap = nn.Sequential(
              nn.AvgPool2d(kernel_size=6, stride=1, padding = 0),#6, 1
        ) 

        ##Conv Block 1
        self.conv_block3 = nn.Sequential(
             nn.Conv2d(in_channels=16, out_channels=16, kernel_size=1, padding=0, bias=False),#6, 4
              nn.ReLU(),
              
              nn.Conv2d(in_channels=16, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),#1, 1
        )

    def forward(self, x):

        x =  self.conv_block1(x)
        x =  self.trans1(x)
        x =  self.conv_block2(x)
        x =  self.gap(x)
        x =  self.conv_block3(x)
        x = x.view(x.size(0), -1)
        return F.log_softmax(x)

##Model Summary:

#### Params used 8,052

In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
            Conv2d-3            [-1, 8, 28, 28]             576
              ReLU-4            [-1, 8, 28, 28]               0
         MaxPool2d-5            [-1, 8, 14, 14]               0
            Conv2d-6           [-1, 10, 12, 12]             720
              ReLU-7           [-1, 10, 12, 12]               0
            Conv2d-8           [-1, 10, 10, 10]             900
              ReLU-9           [-1, 10, 10, 10]               0
           Conv2d-10             [-1, 10, 8, 8]             900
             ReLU-11             [-1, 10, 8, 8]               0
           Conv2d-12             [-1, 16, 6, 6]           1,440
             ReLU-13             [-1, 16, 6, 6]               0
        AvgPool2d-14             [-1, 1

<ipython-input-2-e87865695f25>:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:
model.eval()

Net(
  (conv_block1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (3): ReLU()
  )
  (trans1): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Sequential(
    (0): Conv2d(8, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (3): ReLU()
    (4): Conv2d(10, 10, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (5): ReLU()
    (6): Conv2d(10, 16, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (7): ReLU()
  )
  (gap): Sequential(
    (0): AvgPool2d(kernel_size=6, stride=1, padding=0)
  )
  (conv_block3): Sequential(
    (0): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): ReLU()
    (2): Conv2d(16, 10, kernel_size=(1, 1), stride=(1, 1), bias=False)
 

## Loading DataSet:


In [5]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



## Train Test Functions:

In [6]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    processed = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        processed += len(data)
        pbar.set_description(desc= f'Epoch{epoch} : Loss={loss.item()}  Accuracy={100*correct/processed:0.2f} Batch_id={batch_idx}')
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

    train_loss /= len(train_loader)
    train_acc = 100.00*correct/len(train_loader.dataset)
    return train_acc, train_loss

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    test_acc = 100.00 * correct / len(test_loader.dataset)
    return test_acc, test_loss


## Model Training:

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [ ]:
train_loss_values = []
test_loss_values = []
train_accuracy_values = []
test_accuracy_values = []

In [7]:

for epoch in range(1, 16):
    train_acc, train_loss = train(model, device, train_loader, optimizer, epoch)
    test_acc, test_loss = test(model, device, test_loader)

    train_accuracy_values.append(train_acc)
    train_loss_values.append(train_loss)

    test_accuracy_values.append(test_acc)
    test_loss_values.append(test_loss)

    


  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-e87865695f25>:55: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Epoch1 : Loss=2.302447557449341  Accuracy=9.89 Batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 27.77it/s]



Test set: Average loss: 2.3025, Accuracy: 1032/10000 (10.32%)



Epoch2 : Loss=2.303395986557007  Accuracy=9.91 Batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 31.14it/s]



Test set: Average loss: 2.3019, Accuracy: 1032/10000 (10.32%)



Epoch3 : Loss=2.140258312225342  Accuracy=10.77 Batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.98it/s]


...


Test set: Average loss: 0.0536, Accuracy: 9831/10000 (98.31%)



Epoch14 : Loss=0.033669278025627136  Accuracy=97.99 Batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.85it/s]



Test set: Average loss: 0.0546, Accuracy: 9824/10000 (98.24%)



Epoch15 : Loss=0.05594165250658989  Accuracy=98.17 Batch_id=468: 100%|██████████| 469/469 [00:15<00:00, 30.98it/s]



Test set: Average loss: 0.0467, Accuracy: 9846/10000 (98.46%)

